In [2]:
import os
import sys
import uvicorn
from pathlib import Path
from db import get_connection, close_connection
from base_api import get_api_instance

def insert_data(id: str="test_id", password: str="test_pwd"):
    connection = get_connection()
    try:
        with connection.cursor() as cursor:
            sql = "INSERT INTO users (id, password) VALUES (%s, %s)"
            cursor.execute(sql, (id, password))
        connection.commit()
    finally:
        close_connection(connection)

# markets = ["binance", "okx", "bingx", "bitget"]
# symbols = ["BTCUSDT", "ETHUSDT"]  # Example symbols
# intervals = ["1d", "30m"]   # Example intervals
# start_time='2023-12-01'
# end_time='2024-08-17'
market = "binance"
symbol = "BTCUSDT"
start_time='2023-12-01'
end_time='2024-08-17'
interval = "1d"

api = get_api_instance(market)

data = api.load_data(symbol, interval)
if data is None:
    df = api.get_ohlc_data(symbol=symbol, interval=interval, start_time=start_time, end_time=end_time)
    # api.save_data(df, symbol, interval, start_time, end_time)
    # print(f"Saved data for {market} - {symbol} - {interval}")
else:
    print(f"Data already exists for {market} - {symbol} - {interval}")
# load_data(market="binance", symbol="BTCUSDT", interval=interval, start_time=start_time, end_time=end_time)

Data already exists for binance - BTCUSDT - 1d


ImportError: Missing optional dependency 'pyarrow'.  Use pip or conda to install pyarrow.

In [4]:
pip install pyarrow

   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------- ----------------- 14.2/25.1 MB 80.8 MB/s eta 0:00:01
   ---------------------------------------  24.9/25.1 MB 68.4 MB/s eta 0:00:01
   ---------------------------------------- 25.1/25.1 MB 59.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
